In [26]:
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from mezo.currency_utils import format_pool_token_columns, add_pool_usd_conversions, format_token_columns, add_usd_conversions
from mezo.datetime_utils import format_datetimes
from mezo.data_utils import add_rolling_values, flatten_json_column
from mezo.clients import BigQueryClient, SubgraphClient
from mezo.currency_config import POOLS_MAP, POOL_TOKEN_PAIRS, TOKENS_ID_MAP, TIGRIS_MAP, MEZO_ASSET_NAMES_MAP
from mezo.visual_utils import ProgressIndicators, ExceptionHandler, with_progress
from mezo.queries import PoolQueries

# Helpers

In [27]:
def process_pools_data(raw, transaction_type):
    """Process raw data from musd-pools subgraphs"""
    if not ExceptionHandler.validate_dataframe(raw, "Raw data", ['contractId_', 'timestamp_']):
        raise ValueError("Invalid data structure")
    
    df = raw.copy()
    df['pool'] = df['contractId_'].map(POOLS_MAP)
    df = format_datetimes(df, ['timestamp_'])
    df = format_pool_token_columns(df, 'contractId_', POOL_TOKEN_PAIRS)
    df = add_pool_usd_conversions(df, 'contractId_', POOL_TOKEN_PAIRS, TOKENS_ID_MAP)
    df['transaction_type'] = transaction_type

    return df

In [28]:
def process_volume_data(raw_volume):
    """Process raw volume data from Tigris pools"""
    if not ExceptionHandler.validate_dataframe(raw_volume, "Raw volume data", ['timestamp']):
        raise ValueError("Invalid volume data structure")
    
    df = raw_volume.copy()
    
    # Flatten JSON pool column
    df = flatten_json_column(df, 'pool')
    
    # Format timestamps
    df = format_datetimes(df, ['timestamp'])
    
    # Replace token symbols
    token_columns = ['pool_token0_symbol', 'pool_token1_symbol']
    for col in token_columns:
        df[col] = df[col].replace(MEZO_ASSET_NAMES_MAP)
    
    # Map pool names
    df['pool'] = df['pool_name'].map(TIGRIS_MAP)

    # Format token columns
    df = format_token_columns(df, ['totalVolume0'], 'pool_token0_symbol')
    df = format_token_columns(df, ['totalVolume1'], 'pool_token1_symbol')
    
    # Add USD conversions
    df = add_usd_conversions(df, 'pool_token0_symbol', TOKENS_ID_MAP, ['totalVolume0'])
    df = df.drop(columns=['index', 'usd'])
    df = add_usd_conversions(df, 'pool_token1_symbol', TOKENS_ID_MAP, ['totalVolume1'])
    df = df.drop(columns=['index', 'usd'])
    
    return df


In [ ]:
def process_fees_data(raw_fees):
    """Process raw fees data from Tigris pools"""
    if not ExceptionHandler.validate_dataframe(raw_fees, "Raw fees data", ['timestamp']):
        raise ValueError("Invalid fees data structure")
    
    df = raw_fees.copy()
    
    # Flatten JSON pool column
    df = flatten_json_column(df, 'pool')

    # Format timestamps
    df = format_datetimes(df, ['timestamp'])
    
    # Replace token symbols
    token_columns = ['pool_token0_symbol', 'pool_token1_symbol']

    for col in token_columns:
        df[col] = df[col].replace(MEZO_ASSET_NAMES_MAP)

    # Map pool names
    df['pool'] = df['pool_name'].map(TIGRIS_MAP)
    
    # Format token columns
    df = format_token_columns(df, ['totalFees0'], 'pool_token0_symbol')
    df = format_token_columns(df, ['totalFees1'], 'pool_token1_symbol')
    
    # Add USD conversions
    df = add_usd_conversions(df, 'pool_token0_symbol', TOKENS_ID_MAP, ['totalFees0'])
    df = df.drop(columns=['index', 'usd'])
    df = add_usd_conversions(df, 'pool_token1_symbol', TOKENS_ID_MAP, ['totalFees1'])

    df = df.drop(columns=['index', 'usd', 'pool_token0_symbol', 'pool_token1_symbol'])

    df['total_fees'] = df['totalFees0_usd'] + df['totalFees1_usd']

    df = df[[
        'timestamp', 'pool_name', 'pool', 'totalFees0', 'totalFees1', 
        'totalFees0_usd', 'totalFees1_usd', 'total_fees', 'id'
        ]]
    
    return df

# Get Data

In [130]:
POOLS_SUBGRAPH = "https://api.goldsky.com/api/public/project_cm6ks2x8um4aj01uj8nwg1f6r/subgraphs/musd-pools-mezo/1.1.0/gn"

SUBGRAPH_HEADERS = {
        "Content-Type": "application/json",
    }

In [29]:
# Get volume data
volume_data = SubgraphClient.get_subgraph_data(
    SubgraphClient.TIGRIS_POOLS_SUBGRAPH, 
    PoolQueries.GET_POOL_VOLUME, 
    'poolVolumes'
)

🔍 Trying poolVolumes query...
Fetching transactions with skip=0...
Fetching transactions with skip=1000...
No more records found.
✅ Found 285 poolVolumes records


In [ ]:
# Get deposits data
deposits_data = SubgraphClient.get_subgraph_data(
    POOLS_SUBGRAPH, 
    PoolQueries.GET_DEPOSITS, 
    'mints'
)
# Get withdrawals data
withdrawals_data = SubgraphClient.get_subgraph_data(
    POOLS_SUBGRAPH, 
    PoolQueries.GET_WITHDRAWALS, 
    'burns'
)
# Get fees data
fees_data = SubgraphClient.get_subgraph_data(
    SubgraphClient.TIGRIS_POOLS_SUBGRAPH, 
    PoolQueries.GET_TOTAL_POOL_FEES, 
    'feesStats_collection'
)

🔍 Trying mints query...
Fetching transactions with skip=0...
Fetching transactions with skip=1000...
Fetching transactions with skip=2000...
Fetching transactions with skip=3000...
Fetching transactions with skip=4000...
Fetching transactions with skip=5000...
Fetching transactions with skip=6000...
Fetching transactions with skip=7000...
Fetching transactions with skip=8000...
No more records found.
✅ Found 7527 mints records
🔍 Trying burns query...
Fetching transactions with skip=0...
Fetching transactions with skip=1000...
Fetching transactions with skip=2000...
Fetching transactions with skip=3000...
Fetching transactions with skip=4000...
No more records found.
✅ Found 3603 burns records
🔍 Trying poolVolumes query...
Fetching transactions with skip=0...
Fetching transactions with skip=1000...
No more records found.
✅ Found 278 poolVolumes records
🔍 Trying feesStats_collection query...
Fetching transactions with skip=0...
Fetching transactions with skip=1000...
No more records foun

In [132]:
deposits_data.to_csv('deposits_data.csv')
withdrawals_data.to_csv('withdrawals_data.csv')

In [ ]:
deposits_data = pd.read_csv('deposits_data.csv')
withdrawals_data = pd.read_csv('withdrawals_data.csv')

# Deposits, Withdrawals, TVL

In [143]:
deposits_clean = process_pools_data(deposits_data, 'deposit')
withdrawals_clean = process_pools_data(withdrawals_data, 'withdrawal')

✅ Raw data validation passed (7527 rows)
✅ Raw data validation passed (3603 rows)


In [139]:
deposits_clean.head(10)

,timestamp_,sender,amount0,amount1,contractId_,transactionHash_,pool,token0,token1,token0_index,token1_index,token0_usd_rate,token1_usd_rate,amount0_usd,amount1_usd,transaction_type
0,2025-09-27,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0.000364,39.887861,0x52e604c44417233b6ccedddc0d640a405caacefb,0xa3d2cbf0c1941bff0a12b30d9f7bfed734a7ab50692a...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,109457.0,1.0,39.835831,39.887861,deposit
1,2025-09-27,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0.000329,36.069689,0x52e604c44417233b6ccedddc0d640a405caacefb,0x9705d73be611512cb023b4a42cdbe3d3cf160ea30413...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,109457.0,1.0,36.015621,36.069689,deposit
2,2025-09-27,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0.000439,48.334626,0x52e604c44417233b6ccedddc0d640a405caacefb,0xd749855b3b6c197a93cf01bbcaa5e38139e995e376b7...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,109457.0,1.0,48.022253,48.334626,deposit
3,2025-09-27,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0.000449,49.444921,0x52e604c44417233b6ccedddc0d640a405caacefb,0xf0091714f50f65e26681f6b375964fa4535d2aff28ca...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,109457.0,1.0,49.113801,49.444921,deposit
4,2025-09-27,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0.000698,75.906956,0x52e604c44417233b6ccedddc0d640a405caacefb,0x11f7b48f535c2da8b3ed2717c9e12fabe99e21b445a0...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,109457.0,1.0,76.353912,75.906956,deposit
5,2025-09-27,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0.009955,1083.213150,0x52e604c44417233b6ccedddc0d640a405caacefb,0xb47c889765cdcf5b9f3d3a7038ac8cb8e968636396ea...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,109457.0,1.0,1089.640038,1083.213150,deposit
6,2025-09-27,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0.002694,293.452643,0x52e604c44417233b6ccedddc0d640a405caacefb,0xff91090243573217d41fd43d603d2f2580088531a3f3...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,109457.0,1.0,294.856866,293.452643,deposit
7,2025-09-27,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0.000499,54.389472,0x52e604c44417233b6ccedddc0d640a405caacefb,0x75dfecb3472c6d8874e1c93c61c9fde8f70ebdb4d36c...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,109457.0,1.0,54.571593,54.389472,deposit
8,2025-09-27,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0.001795,195.937564,0x52e604c44417233b6ccedddc0d640a405caacefb,0x0194e2a5ed7acd61c204c27465dc63b9a476a6921fc7...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,109457.0,1.0,196.525960,195.937564,deposit
9,2025-09-27,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0.000309,33.760518,0x52e604c44417233b6ccedddc0d640a405caacefb,0x2f238cfcb19e88e5176baa5d6923a11f866ac71361df...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,109457.0,1.0,33.829367,33.760518,deposit


In [140]:
withdrawals_clean.head()
withdrawals_clean.to_csv('wits.csv')
deposits_clean.to_csv('dep.csv')

In [144]:
deposits = deposits_clean.copy()
withdrawals = withdrawals_clean.copy()

# Combine deposits and withdrawals
combined = pd.concat([deposits, withdrawals], ignore_index=True)
    
# Sort by timestamp for proper cumulative calculations
combined = combined.sort_values('timestamp_').reset_index(drop=True)

combined

,timestamp_,sender,amount0,amount1,contractId_,transactionHash_,pool,token0,token1,token0_index,token1_index,token0_usd_rate,token1_usd_rate,amount0_usd,amount1_usd,transaction_type,to
0,2025-05-28,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0.000921,99.999999,0x9e60cd4d5b718178fab0137200a36a5472191302,0xd9bad9a4bcdebbe51b770b978e48378c89faf8d7cc52...,btc_musdt_pool,BTC,USDT,bitcoin,tether,114341.000000,1.001,105.325165,100.099999,withdrawal,0x52d033e64774f31a8b7562806aa5782cbbd29382
1,2025-05-28,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0.000921,100.000000,0x52e604c44417233b6ccedddc0d640a405caacefb,0xa2e26a6832aaa7b0dcd39dd876c545b1bd0fd5f061f0...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,114341.000000,1.000,105.325166,100.000000,deposit,NaN
2,2025-05-28,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,100.000000,0.000921,0xb07c2172c4b7bbc3ac52088d30cb103853b0b403,0xf0792943728e53e725da231295197e6dd674fc866234...,musdc_btc_pool,USDC,BTC,usd-coin,bitcoin,0.999613,114341.000,99.961300,105.325166,deposit,NaN
3,2025-05-28,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,100.000000,100.000000,0x2a1ab0224a7a608d3a992cb15594a2934f74f4c0,0x371f560ed62a94280ad87c69911d18b9369524c4ceb4...,musdc_musdt_pool,USDC,mUSDT,usd-coin,NaN,0.999613,NaN,99.961300,NaN,deposit,NaN
4,2025-05-28,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0.000921,100.000000,0x9e60cd4d5b718178fab0137200a36a5472191302,0x56c593133b319f5dcb1bae7d3cc48d784dc90e420b00...,btc_musdt_pool,BTC,USDT,bitcoin,tether,114341.000000,1.001,105.325166,100.100000,deposit,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11125,2025-09-27,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,113.132479,159.192986,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0xeb7da60650a787d853ed7d622b1a8150de06c38604f3...,musdc_musd_pool,USDC,MUSD,usd-coin,NaN,0.999613,1.000,113.088697,159.192986,withdrawal,0x0dbbc9d376294b63b631719c15b67362f105692a
11126,2025-09-27,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,2.838078,3.992015,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0x1d958a20dfedfe4b3a0f7271b501b93caf302805a1b6...,musdc_musd_pool,USDC,MUSD,usd-coin,NaN,0.999613,1.000,2.836980,3.992015,withdrawal,0x6ab8a95dd281b10bb7a8c5a40e88a4f253fa8690
11127,2025-09-27,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0.000045,4.896609,0x52e604c44417233b6ccedddc0d640a405caacefb,0x7823e4ac5d48442c950367349f4076eb53749283cc0c...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,114341.000000,1.000,5.100202,4.896609,withdrawal,0xc492cbb19aa66b829ba5ee42fb94e73bc724a6f8
11128,2025-09-27,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0.001596,174.336797,0x52e604c44417233b6ccedddc0d640a405caacefb,0xd4c31ab9188b8fb68795a989ccedd01574798fa92143...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,114341.000000,1.000,182.474780,174.336797,deposit,NaN


In [145]:
# Calculate net amounts (deposits positive, withdrawals negative)
combined['net_amount0_usd'] = np.where(
    combined['transaction_type'] == 'deposit',
    combined['amount0_usd'],
    -combined['amount0_usd']
)

combined['net_amount1_usd'] = np.where(
    combined['transaction_type'] == 'deposit',
    combined['amount1_usd'],
    -combined['amount1_usd']
)

combined['net_total_usd'] = combined['net_amount0_usd'] + combined['net_amount1_usd']

# Calculate absolute amounts for deposits/withdrawals tracking
combined['deposit_amount_usd'] = np.where(
    combined['transaction_type'] == 'deposit',
    combined['amount0_usd'] + combined['amount1_usd'],
    0
)

combined['withdrawal_amount_usd'] = np.where(
    combined['transaction_type'] == 'withdrawal',
    combined['amount0_usd'] + combined['amount1_usd'],
    0
)

# Calculate cumulative TVL for each pool
combined['tvl_token0_usd'] = combined.groupby('pool')['net_amount0_usd'].cumsum()
combined['tvl_token1_usd'] = combined.groupby('pool')['net_amount1_usd'].cumsum()
combined['tvl_total_usd'] = combined['tvl_token0_usd'] + combined['tvl_token1_usd']

# Get token information for each pool
token_info = combined.groupby('pool').agg({
    'token0': 'first',
    'token1': 'first'
}).to_dict('index')

In [149]:
combined['num_withdrawals'] = np.where(
    combined['transaction_type'] == 'withdrawal',
    1,
    0
)

In [150]:
combined['num_withdrawals']

0        1
1        0
2        0
3        0
4        0
        ..
11125    1
11126    1
11127    1
11128    0
11129    0
Name: num_withdrawals, Length: 11130, dtype: int64

In [147]:
combined[['timestamp_', 'pool', 'net_amount0_usd', 'net_amount1_usd', 
       'net_total_usd', 'deposit_amount_usd', 'withdrawal_amount_usd', 
       'tvl_token0_usd', 'tvl_token1_usd', 'tvl_total_usd', 'sender', 'to']]

,timestamp_,pool,net_amount0_usd,net_amount1_usd,net_total_usd,deposit_amount_usd,withdrawal_amount_usd,tvl_token0_usd,tvl_token1_usd,tvl_total_usd,sender,to
0,2025-05-28,btc_musdt_pool,-105.325165,-100.099999,-205.425164,0.000000,205.425164,-1.053252e+02,-100.099999,-205.425164,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x52d033e64774f31a8b7562806aa5782cbbd29382
1,2025-05-28,btc_musd_pool,105.325166,100.000000,205.325166,205.325166,0.000000,1.053252e+02,100.000000,205.325166,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,NaN
2,2025-05-28,musdc_btc_pool,99.961300,105.325166,205.286466,205.286466,0.000000,9.996130e+01,105.325166,205.286466,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,NaN
3,2025-05-28,musdc_musdt_pool,99.961300,NaN,NaN,NaN,0.000000,9.996130e+01,NaN,NaN,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,NaN
4,2025-05-28,btc_musdt_pool,105.325166,100.100000,205.425166,205.425166,0.000000,3.470301e-07,0.000001,0.000001,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
11125,2025-09-27,musdc_musd_pool,-113.088697,-159.192986,-272.281683,0.000000,272.281683,2.585587e+05,200600.270886,459158.987724,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x0dbbc9d376294b63b631719c15b67362f105692a
11126,2025-09-27,musdc_musd_pool,-2.836980,-3.992015,-6.828994,0.000000,6.828994,2.585559e+05,200596.278871,459152.158730,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x6ab8a95dd281b10bb7a8c5a40e88a4f253fa8690
11127,2025-09-27,btc_musd_pool,-5.100202,-4.896609,-9.996810,0.000000,9.996810,4.196297e+05,415981.058045,835610.765218,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0xc492cbb19aa66b829ba5ee42fb94e73bc724a6f8
11128,2025-09-27,btc_musd_pool,182.474780,174.336797,356.811577,356.811577,0.000000,4.198122e+05,416155.394842,835967.576795,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,NaN


# Volume Calculations

In [30]:
volume_clean = process_volume_data(volume_data)

✅ Raw volume data validation passed (285 rows)


In [31]:
volume_clean

,timestamp,totalVolume0,totalVolume1,id,pool_name,pool_token0_symbol,pool_token1_symbol,pool,totalVolume0_usd,totalVolume1_usd
0,2025-09-28,0.028041,3169.057728,15099326896275457,Volatile AMM - BTC/MUSD,BTC,MUSD,btc_musd_pool,3266.753855,3169.057728
1,2025-09-28,105.155806,114.031500,15099223817060353,Stable AMM - MUSD/mUSDT,MUSD,USDT,musd_musdt_pool,105.155806,114.031500
2,2025-09-28,1604.706289,309.367520,15094851540353025,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool,1604.171922,309.367520
3,2025-09-27,0.081300,8360.241291,14998300675538945,Volatile AMM - BTC/MUSD,BTC,MUSD,btc_musd_pool,9471.414615,8360.241291
4,2025-09-27,1025.454941,154.880104,14985175255482369,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool,1025.113465,154.880104
...,...,...,...,...,...,...,...,...,...,...
280,2025-07-08,0.000250,0.000000,6679932570697729,Volatile AMM - BTC/MUSD,BTC,MUSD,btc_musd_pool,29.125000,0.000000
281,2025-07-07,0.000000,18.910000,6572738776924161,Volatile AMM - BTC/MUSD,BTC,MUSD,btc_musd_pool,0.000000,18.910000
282,2025-07-07,2.010400,0.000000,6572554093330433,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool,2.009731,0.000000
283,2025-06-26,0.000000,1.000000,5430148627169281,Stable AMM - MUSD/mUSDT,MUSD,USDT,musd_musdt_pool,0.000000,1.000000


In [32]:
# calculate the total volume for each row

def get_volume_for_row(row):
    """Determine which volume to use based on token types."""
    volatiles = ['SolvBTC', 'xSolvBTC']
    stables = ['USDC', 'USDT', 'upMUSD']

    token0 = row['pool_token0_symbol']
    token1 = row['pool_token1_symbol']
    vol0 = row['totalVolume0_usd']
    vol1 = row['totalVolume1_usd']

    # priority 1: MUSD
    if token0 == 'MUSD':
        return vol0
    elif token1 == 'MUSD':
        return vol1
    # priority 2: other stables
    elif token0 in stables and token1 not in stables:
        return vol0
    elif token1 in stables and token0 not in stables:
        return vol1
    # priority 3: take BTC if pair is another volatile asset
    elif token0 == 'BTC' and token1 in volatiles:
        return vol0
    elif token1 == 'BTC' and token0 in volatiles:
        return vol1
    else:
        return max(vol0, vol1)

# Apply to each row
volume_clean['total_volume'] = volume_clean.apply(get_volume_for_row, axis=1)

In [33]:
df = volume_clean.copy()

df['timestamp'] = pd.to_datetime(df['timestamp'])
df['date'] = df['timestamp'].dt.date

In [34]:
df[['date', 'pool', 'total_volume']]

,date,pool,total_volume
0,2025-09-28,btc_musd_pool,3169.057728
1,2025-09-28,musd_musdt_pool,105.155806
2,2025-09-28,musdc_musd_pool,309.367520
3,2025-09-27,btc_musd_pool,8360.241291
4,2025-09-27,musdc_musd_pool,154.880104
...,...,...,...
280,2025-07-08,btc_musd_pool,0.000000
281,2025-07-07,btc_musd_pool,18.910000
282,2025-07-07,musdc_musd_pool,0.000000
283,2025-06-26,musd_musdt_pool,0.000000


In [40]:
daily_volume = df.groupby(['date', 'pool']).agg(
    volume0 = ('totalVolume0_usd', 'last'),
    volume1 = ('totalVolume1_usd', 'last'),
    total_volume = ('total_volume', 'last')
).reset_index().sort_values('date', ascending=False).fillna(0)

In [41]:
daily_volume

,date,pool,volume0,volume1,total_volume
284,2025-09-28,musdc_musd_pool,1604.171922,309.367520,309.367520
283,2025-09-28,musd_musdt_pool,105.155806,114.031500,105.155806
282,2025-09-28,btc_musd_pool,3266.753855,3169.057728,3169.057728
281,2025-09-27,musdc_musdt_pool,134.982042,0.000000,134.982042
280,2025-09-27,musdc_musd_pool,1025.113465,154.880104,154.880104
...,...,...,...,...,...
4,2025-07-08,btc_musd_pool,29.125000,0.000000,0.000000
2,2025-07-07,btc_musd_pool,0.000000,18.910000,18.910000
3,2025-07-07,musdc_musd_pool,2.009731,0.000000,0.000000
1,2025-06-26,musd_musdt_pool,0.000000,1.000000,0.000000


In [43]:
daily_volume_all = daily_volume.groupby('date').agg({
    'volume0': 'sum',
    'volume1': 'sum',
    'total_volume': 'sum'
})

In [50]:
daily_volume_all = daily_volume_all.sort_values(by='date', ascending=False)

In [66]:
from datetime import datetime, date
daily_volume.loc[daily_volume['date'] >= datetime(2025, 9, 25).date()]

,date,pool,volume0,volume1,total_volume
284,2025-09-28,musdc_musd_pool,1604.171922,309.367520,309.367520
283,2025-09-28,musd_musdt_pool,105.155806,114.031500,105.155806
282,2025-09-28,btc_musd_pool,3266.753855,3169.057728,3169.057728
281,2025-09-27,musdc_musdt_pool,134.982042,0.000000,134.982042
280,2025-09-27,musdc_musd_pool,1025.113465,154.880104,154.880104
279,2025-09-27,musd_musdt_pool,45.758800,154.720391,45.758800
278,2025-09-27,btc_musd_pool,9471.414615,8360.241291,8360.241291
277,2025-09-26,musdc_musd_pool,474.603627,598.122799,598.122799
276,2025-09-26,btc_musd_pool,4826.528404,5825.347217,5825.347217
275,2025-09-25,musdc_musdt_pool,651.468928,769.059306,769.059306


In [69]:
volume_clean.loc[volume_clean['timestamp'] >= datetime(2025, 9, 25).date()].to_csv('volume_data-09252025.csv')

In [44]:
# Create separate dataframes for each pool
pools = {}

# Get unique pools
unique_pools = daily_volume['pool'].unique()
print(f"Found {len(unique_pools)} unique pools: {list(unique_pools)}")

# Create a separate dataframe for each pool
for pool_name in unique_pools:
    if pd.notna(pool_name):  # Skip NaN values if any
        pool_df = daily_volume[daily_volume['pool'] == pool_name].copy()
        pools[pool_name] = pool_df
        print(f"{pool_name}: {len(pool_df)} rows")

# Access individual pool dataframes like this:

pools


Found 5 unique pools: ['musdc_musd_pool', 'musd_musdt_pool', 'btc_musd_pool', 'musdc_musdt_pool', 'musdc_btc_pool']
musdc_musd_pool: 78 rows
musd_musdt_pool: 75 rows
btc_musd_pool: 84 rows
musdc_musdt_pool: 47 rows
musdc_btc_pool: 1 rows


{'musdc_musd_pool':            date             pool       volume0        volume1   total_volume
 284  2025-09-28  musdc_musd_pool   1604.171922     309.367520     309.367520
 280  2025-09-27  musdc_musd_pool   1025.113465     154.880104     154.880104
 277  2025-09-26  musdc_musd_pool    474.603627     598.122799     598.122799
 274  2025-09-25  musdc_musd_pool  25617.390805  110549.180573  110549.180573
 271  2025-09-24  musdc_musd_pool   5794.189723    2315.415713    2315.415713
 ..          ...              ...           ...            ...            ...
 20   2025-07-17  musdc_musd_pool   4968.047627    3796.893018    3796.893018
 17   2025-07-16  musdc_musd_pool   6219.141505    6036.966709    6036.966709
 11   2025-07-11  musdc_musd_pool      7.001183       2.003906       2.003906
 7    2025-07-09  musdc_musd_pool      0.000000       2.009900       2.009900
 3    2025-07-07  musdc_musd_pool      2.009731       0.000000       0.000000
 
 [78 rows x 5 columns],
 'musd_musdt_pool':

In [45]:
keys = pools.keys()
from mezo.data_utils import add_pct_change_columns
for key in keys:
    pools[key] = add_rolling_values(pools[key], 7, ['volume0', 'volume1', 'total_volume'])
    pools[key] = add_rolling_values(pools[key], 30, ['volume0', 'volume1', 'total_volume'])
    pools[key]['is_significant_volume'] = pools[key]['total_volume'].transform(lambda x: x > x.quantile(0.9))
    pools[key] = add_pct_change_columns(pools[key], ['volume0', 'volume1', 'total_volume'], 'daily').fillna(0)

# Fee Calculations

In [119]:
fees_clean = process_fees_data(fees_data)

✅ Raw fees data validation passed (276 rows)


In [120]:
fees_clean

,timestamp,pool_name,pool,totalFees0,totalFees1,totalFees0_usd,totalFees1_usd,total_fees,id
0,2025-09-25,Volatile AMM - BTC/MUSD,btc_musd_pool,4.697195e-04,41.715310,51.418789,41.715310,93.134100,14795067118059520
1,2025-09-25,Stable AMM - mUSDC/MUSD,musdc_musd_pool,5.125166e+00,22.109836,5.123157,22.109836,27.232993,14794882434465792
2,2025-09-25,Stable AMM - MUSD/mUSDT,musd_musdt_pool,1.576448e-01,0.022591,0.157645,0.022614,0.180258,14777122744696832
3,2025-09-25,Stable AMM - mUSDC/mUSDT,musdc_musdt_pool,1.303360e-01,0.153810,0.130285,0.153964,0.284249,14770590099439618
4,2025-09-24,Volatile AMM - BTC/MUSD,btc_musd_pool,7.734394e-04,76.967154,84.666090,76.967154,161.633244,14693495436476416
...,...,...,...,...,...,...,...,...,...
271,2025-07-08,Volatile AMM - BTC/MUSD,btc_musd_pool,7.500000e-07,0.000000,0.082100,0.000000,0.082100,6679932570697728
272,2025-07-07,Volatile AMM - BTC/MUSD,btc_musd_pool,0.000000e+00,0.056730,0.000000,0.056730,0.056730,6572738776924160
273,2025-07-07,Stable AMM - mUSDC/MUSD,musdc_musd_pool,4.020000e-04,0.000000,0.000402,0.000000,0.000402,6572554093330432
274,2025-06-26,Stable AMM - MUSD/mUSDT,musd_musdt_pool,0.000000e+00,0.000100,0.000000,0.000100,0.000100,5430148627169280
